<a href="https://colab.research.google.com/github/jvishnuvardhan/Stackoverflow_Questions/blob/master/Multiloss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nig

In [0]:
import tensorflow as tf

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import keras
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from resnet3d import Resnet3DBuilder
import resnet
import keras

              
data_path_l ='/content/gdrive/My Drive/Data/training_left'
data_path_r ='/content/gdrive/My Drive/Data/training_right'

data_path_dip ='/content/gdrive/My Drive/Data/training_disp'

num_classes=7
batch_size=16
epochs=200
train_images=274


img_channels=3
img_rows=224
img_cols=224


input_imgen = ImageDataGenerator(validation_split=0.2)

def generate_generator_multiple(generator,dir1, dir2, dir3, batch_size):
    genX1 = generator.flow_from_directory(directory=dir1,target_size=(img_rows,img_cols),
                                          color_mode="rgb",
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=True, seed=42,subset='training')
                                          
      
    genX2 = generator.flow_from_directory(directory=dir2,target_size=(img_rows,img_cols),
                                          color_mode="rgb",
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=True, seed=42,subset='training')
    
    genX3 = generator.flow_from_directory(directory=dir3,target_size=(img_rows,img_cols),
                                          color_mode="rgb",
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=True, seed=42,subset='training')
    
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()

            Xsum = np.concatenate((X1i[0],X2i[0]), axis=3)
            Xsum = np.expand_dims(Xsum, axis=1)
            Xsum=np.swapaxes(Xsum,4,1)

            yield [Xsum,X3i[0]], [X2i[1],X3i[1]]  #Yield both images and their mutual label
            
             
 
           
inputgenerator=generate_generator_multiple(generator=input_imgen,
                                           dir1=data_path_l,
                                           dir2=data_path_r,
                                           dir3=data_path_dip,
                                           batch_size=batch_size)


def multi_loss_model():
    model_rgb  = Resnet3DBuilder.build_resnet_50((6,img_rows,img_cols,1), num_classes)
    model_disp = resnet.ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), num_classes)
    custom_resnet_model = Model(inputs=[model_rgb.input,model_disp.input], outputs=[model_rgb.output,model_disp.output])

    return custom_resnet_model




model_resnet_multi_class= multi_loss_model()
model_resnet_multi_class.compile(optimizer=keras.optimizers.Adam(),
                                 loss=[keras.losses.CategoricalCrossentropy(), keras.losses.CategoricalCrossentropy()],
                                  loss_weights=[1., 0.2],
                                 metrics=['accuracy'])
    


history=model_resnet_multi_class.fit_generator(inputgenerator,
                       steps_per_epoch= train_images // batch_size,
                        epochs = epochs)




AttributeError: ignored